In [ ]:
"""
Goal: hourly power breakdown for South Korea from Jan 1st, 2019 to December 31st, 2019

Output csv format:

datetime, 
timestamp,
zone_name,
total_consumption_avg,
power_production_biomass_avg,
power_production_coal_avg,
power_production_gas_avg,
power_production_hydro_avg,
power_production_nuclear_avg,
power_production_oil_avg,
power_production_solar_avg,
power_production_wind_avg,
power_production_geothermal_avg,
power_production_unknown_avg
"""


In [1]:
from datetime import datetime

In [13]:
def convert_datetime_to_int(datetime_obj):
    """
    Datetime object has the following format:
    datetime(year, month, day, hour, minute, second, microsecond)
    """
    org_datetime = datetime(1970,1,1,0,0,0,0)
    timestamp = int((datetime_obj-org_datetime).total_seconds())
    return timestamp

In [17]:
convert_datetime_to_int(datetime(2018,1,1,0,0,0,0))

1514764800